## path 1:
create data folber


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path
from vision.Config import dir_dataset_folder_val, dir_dataset_folder

## path 1 
dir_folber_val = Path(dir_dataset_folder_val , 'inter folder dataset ')
dir_folber = Path(dir_dataset_folder , 'inter folder dataset ')
from vision.Create_Dataset import Create_dataset
from vision.Config import dic_dir_val,dic_dir,dir_dataset_folder,dir_dataset_folder_val, Create_train_data, Create_val_data
from pathlib import Path


if Create_train_data:
    id_class = 1
    dic_name_folber_clases = {
        
        f'class_id{id_class}':['core_class' ,'cut_image' , 'box_image'],
        'calss_other':['non_core_class', 'bake_image_Not']

        }

    dir_folber = Path(dir_dataset_folder , 'extended_dataset_all_folderes')

    Create_dataset (dic_name = dic_name_folber_clases ,
                         dic_dir = dic_dir,
                        dir_extended = dir_folber,
                        move_or_copy_files = 'move'
                        )
    

if Create_val_data:
    id_class = 1
    dic_name_folber_clases_val = {
        
        f'class_id{id_class}':['core_class' ,'cut_image' , 'box_image'],
        'calss_other':['non_core_class', 'bake_image_Not']

        }

    dir_folber_val = Path(dir_dataset_folder_val , 'extended_dataset_all_folderes')

    Create_dataset (dic_name = dic_name_folber_clases_val ,
                        dic_dir = dic_dir_val,
                        dir_extended = dir_folber_val,
                        move_or_copy_files = 'move'
                        )
    
    dic_name_folber_clases_val = {
        
        f'class_id{id_class}':['core_class' ],
        'calss_other':[ 'bake_image_Not']

        }

    dir_folber_val_1 = Path(dir_dataset_folder_val , 'extended_dataset_bakeImageNot_and_coreClass_folderes')

    Create_dataset (dic_name = dic_name_folber_clases_val ,
                        dic_dir = dic_dir_val,
                        dir_extended = dir_folber_val_1,
                        move_or_copy_files = 'move'
                        )


## path 2:
Create dataset transform ...

In [ ]:
## path 2
# 4. آماده‌سازی داده‌ها
transform = transforms.Compose([
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),   # تغییر اندازه به ورودی مدل
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


transform_val = transforms.Compose([
    transforms.Resize((224, 224)),   # تغییر اندازه به ورودی مدل
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

BATCH_SIZE = 2 ** 7
# ImageFolder دو کلاس را 0 و 1 نگاشت می‌کند
dataset_val = datasets.ImageFolder(root=dir_folber_val, transform=transform_val)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

print(dataset_val.class_to_idx)

# e.g. {'negative': 0, 'positive': 1}
# ImageFolder دو کلاس را 0 و 1 نگاشت می‌کند
dataset_train = datasets.ImageFolder(root=dir_folber, transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

print(dataset_train.class_to_idx)


## path 3:

In [ ]:
## path 3
from torch import nn
from torchvision import models

# 1. بارگذاری مدل پیش‌آموزش‌دیده SqueezeNet
model_3 = models.squeezenet1_1(pretrained=True)

# 2. تغییر لایه آخر مدل به یک نود (برای باینری: وجود یا عدم وجود انسان)
model_3 .classifier[1] = nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))

# اضافه کردن Softmax برای تبدیل خروجی به احتمال باینری
# این مرحله در اینجا مورد نیاز نیست، چون BCEWithLogitsLoss به‌طور داخلی سیگموید را اعمال می‌کند.

model_3 .classifier = nn.Sequential(
    model_3 .classifier[0],
    model_3 .classifier[1],
    nn.Conv2d(in_channels=1, out_channels=1, kernel_size=13, stride=1, padding=0),
    nn.Flatten()
)
for param in model_3 .features[:10].parameters():
    param.requires_grad = False  # غیرفعال کردن آموزش برای لایه‌های اولیه



